In [56]:
from copy import copy
import glob
import hashlib
import json
import os
from pathlib import Path
import shutil

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image, ImageChops, ImageFile, ImageFilter
from tqdm import tqdm_notebook as tqdm

ImageFile.LOAD_TRUNCATED_IMAGES = True
plt.style.use('dark_background')

# Define functions

In [2]:
def trim_stage1(img):
    img_array = np.asarray(img)
    vertical_crop = img_array[:,:5,:]
    vertical_means = np.mean(vertical_crop, axis=2).mean(axis=1)
    diffs = []
    for i,pixel in enumerate(vertical_means):
        diff = vertical_means[i] - vertical_means[i-1]
        diffs.append(diff)
    crop_max = np.max(diffs[256:])
    crop_min = np.min(diffs[:256])
    max_ix = np.argmax(diffs[256:])
    min_ix = np.argmin(diffs[:256])
    crop_mean = np.mean(diffs)
    new_array = img_array[min_ix:256+max_ix,:,:]
    new_img = Image.fromarray(new_array, 'RGB')
    return new_img

def trim_stage2(img):
    bg = Image.new(img.mode, img.size, img.getpixel((0,0)))
    diff = ImageChops.difference(img, bg)
    diff = ImageChops.add(diff, diff, 2.0, -100)
    bbox = diff.getbbox()
    if bbox:
        return img.crop(bbox)
    else:
        pass

def is_square(img):
    height = np.shape(img)[0]
    width = np.shape(img)[1]
    if 0.3 < (height/width) < 1.2:
        return True
    else:
        return False
    
def makeShadow(image, iterations, border, offset, backgroundColour='black', shadowColour='black'):
    # image: base image to give a drop shadow
    # iterations: number of times to apply the blur filter to the shadow
    # border: border to give the image to leave space for the shadow
    # offset: offset of the shadow as [x,y]
    # backgroundCOlour: colour of the background
    # shadowColour: colour of the drop shadow
    
    #Calculate the size of the shadow's image
    fullWidth  = image.size[0] + abs(offset[0]) + 2*border
    fullHeight = image.size[1] + abs(offset[1]) + 2*border
    
    #Create the shadow's image. Match the parent image's mode.
    shadow = Image.new(image.mode, (fullWidth, fullHeight), backgroundColour)
    
    # Place the shadow, with the required offset
    shadowLeft = border + max(offset[0], 0) #if <0, push the rest of the image right
    shadowTop  = border + max(offset[1], 0) #if <0, push the rest of the image down
    #Paste in the constant colour
    shadow.paste(shadowColour, 
                [shadowLeft, shadowTop,
                 shadowLeft + image.size[0],
                 shadowTop  + image.size[1] ])
    
    # Apply the BLUR filter repeatedly
    for i in range(iterations):
        shadow = shadow.filter(ImageFilter.BLUR)

    # Paste the original image on top of the shadow 
    imgLeft = border - min(offset[0], 0) #if the shadow offset was <0, push right
    imgTop  = border - min(offset[1], 0) #if the shadow offset was <0, push down
    shadow.paste(image, (imgLeft, imgTop))

    return shadow


def cylindricalWarp(img):
    """This function returns the cylindrical warp for a given image and intrinsics matrix K"""
    img = np.array(img)
    h_,w_ = img.shape[:2]
    K = np.array([[600,0,w_/2],[0,600,h_/2],[0,0,1]])
    # pixel coordinates
    y_i, x_i = np.indices((h_,w_))
    X = np.stack([x_i,y_i,np.ones_like(x_i)],axis=-1).reshape(h_*w_,3) # to homog
    Kinv = np.linalg.inv(K) 
    X = Kinv.dot(X.T).T # normalized coords
    # calculate cylindrical coords (sin\theta, h, cos\theta)
    A = np.stack([np.sin(X[:,0]),X[:,1],np.cos(X[:,0])],axis=-1).reshape(w_*h_,3)
    B = K.dot(A.T).T # project back to image-pixels plane
    # back from homog coords
    B = B[:,:-1] / B[:,[-1]]
    # make sure warp coords only within image bounds
    B[(B[:,0] < 0) | (B[:,0] >= w_) | (B[:,1] < 0) | (B[:,1] >= h_)] = -1
    B = B.reshape(h_,w_,-1)
    
    img_rgba = cv2.cvtColor(img,cv2.COLOR_BGR2BGRA) # for transparent borders...
    
    # warp the image according to cylindrical coords
    img_warped = cv2.remap(
        img_rgba, 
        B[:,:,0].astype(np.float32), 
        B[:,:,1].astype(np.float32), 
        cv2.INTER_AREA, 
        borderMode=cv2.BORDER_TRANSPARENT
    )
    return img_warped


## Run Loop

'02f9535f1c946be82e9694e78e7bc4ae'

In [65]:
# Set paths
blur_dir = "/mnt/h/data/stylegan2_ada_outputs/"
dest_dir = "/mnt/h/data/stylegan2_ada_outputs/bottled/"
bottles_path = "/mnt/c/Users/david/Documents/github/this-wine-does-not-exist/images/bottle_templates.jpg"

Path(dest_dir).mkdir(exist_ok=True)

# Load blur results JSON
blur_results = []
for i in Path(blur_dir).glob('blur_results_*.json'):
    with open(i, 'rb') as f:
        r = json.load(f)
    [blur_results.append(i) for i in r['results']]

# Load bottle templates
bottles_templates = Image.open(bottles_path)
dark_bottle = bottles_templates.crop(box=(0,0,1000,3000)).resize((593, 1750), Image.ANTIALIAS)

shift = -8
light_bottle = bottles_templates.crop(box=(2000+shift,0,3000+shift,3000)).resize((593, 1750), Image.ANTIALIAS)
#dark_bottle.show()
#light_bottle.show()

dark_wine_cats = [14,2,8,1,5,4,9,12,15]
light_wine_cats = [3,13,7,6,10]
display_imgs = False
left = 51
upper = 950
label_size = 512


# Main loop
ixx = 0
for i in tqdm(blur_results):
    try:
        if i['score'] < 3:
            continue
        # Load image
        img_Path = Path(i['input_path'])
        raw_img = Image.open(img_Path)

        # Trim/Crop image
        #img = copy(raw_img)
        #img = trim_stage1(raw_img)
        img = trim_stage2(raw_img)
        if is_square(img) == False:
            continue

        img3 = img.resize((label_size,label_size), Image.ANTIALIAS)

        # Warp image for bottle wrapping
        img_warped = cylindricalWarp(img3)
        img_mask = img_warped[:, :, 3] < 255
        img_warped[:, :, 3][img_mask] = 0
        img4 = Image.fromarray(img_warped)

        # Make shadow
        #img = makeShadow(img, iterations=1, border=0, offset=(3,3))

        cat_2 = int(str(img_Path.parents[0]).split('9480_')[1])
        if cat_2 not in dark_wine_cats + light_wine_cats:
            continue

        new_bottle = copy(dark_bottle) if cat_2 in dark_wine_cats else copy(light_bottle)
        new_bottle.paste(im=img4, box=(left, upper, left+label_size, upper+label_size), mask=img4)
        new_bottle = new_bottle.resize((300,900), Image.ANTIALIAS)

        if display_imgs == True:
            fig, ax = plt.subplots(1,3, figsize=(15,5))
            ax[0].imshow(raw_img);
            ax[2].imshow(img)
            ax[3].imshow(new_bottle)
            plt.show()

        # Write bottles with labels
        dest_file = f"bottle_cat_{cat_2}_seed_{ixx:05d}.png"
        new_bottle.save(Path(dest_dir, dest_file))

        #if ixx == 20:
        #    break
        ixx += 1
    except Exception as e:
        print(Exception)


<ipython-input-65-b5677c8263f5>:34: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(blur_results):


  0%|          | 0/15015 [00:00<?, ?it/s]